In [1]:
# NN to recognize hand-written digits using the MNIST data
using DelimitedFiles
using StatsBase
using Distributions
using LinearAlgebra

# read the MNIST data
const testx = readdlm("testx.csv", ',', Int, '\n')
const testy = readdlm("testy.csv", ',', Int, '\n')
const trainx = readdlm("trainx.csv", ',', Int, '\n')
const trainy = readdlm("trainy.csv", ',', Int, '\n')

const L = 3                 # number of layers including input and output
const sizes = [784, 30, 10] # number of neurons in each layer

@. f(z) = 1/(1 + exp(-z))      # sigmoid activation
@. fprime(z) = f(z) * (1-f(z))

# convert a digit d to a 10-element vector
# e.g. 6 is converted to [0,0,0,0,0,0,1,0,0,0]
function digit2vector(d)
    vcat( repeat([0], d), 1, repeat([0], 9-d) )
end

# a feedforward function that returns the activations
# from each layer and the weighted inputs to each layer
# so that they can be used during backpropagation.
# W,b contain the weights, biases in the network.
# x is the input of a single training example (a vector of length 784).
function feedforward(W, b, x)
    a = [x,zeros(30), zeros(10)]
    z = [x, zeros(30), zeros(10)]
    z[2] = W[1] * x + b[1]
    a[2] = f.(z[2])
    z[3] = W[2] * a[2] + b[2]
    a[3] = f.(z[3])
    return a, z
end

# given an input vector, return the predicted digit
function classify(W, b, x)
    find_max = findmax(x)[2]
    return find_max
end

# helper function for backprop().
# this function computes the error for a single training example.
# W contains the weights in the network.
# a contains the activations.
# z contains the weighted inputs.
# y is the correct digit.
# returns δ = the error. the size of δ is [ 784, 30, 10 ]
function compute_error(W, a, z, y)
    δ = [ zeros(sizes[1]), zeros(sizes[2]), zeros(sizes[3]) ]
    # note that δ[1] is junk. we put it there so that the indices make sense.

    # at the output layer L
    δ[3] = -(digit2vector(y) .- a[3]) .* fprime(z[3])

    # for each earlier layer L-1,L-2,..,2 (for the HW, this means only layer 2)
    δ[2] = W[2]' * δ[3] .* fprime(z[2])

    return δ
end

# helper function for backprop(). given the errors δ and the
# activations a for a single training example, this function returns
# the gradient components ∇W and ∇b.
# this function implements the equations BP3 and BP4.
function compute_gradients(δ, a)

    ∇W = δ[3] .* transpose(a[3])
    ∇b = δ[3]
    return ∇W, ∇b
end

# backpropagation. returns ∇W and ∇b for a single training example.
function backprop(W, b, x, y)
    (a, z) = feedforward(W, b, x)
    δ = compute_error(W, a, z, y)
    (∇W, ∇b) = compute_gradients(δ, a)
    return ∇W, ∇b
end

# gradient descent algorithm.
# W = weights in the network
# b = biases in the network
# batch = the indices of the observations in the batch, i.e. the rows of trainx
# α = step size
# λ = regularization parameter

function GD(W, b, batch; α=0.01, λ=0.01)
    m = length(batch)    # batch size
    
    # data structure to accumulate the sum over the batch.
    # in the notes and in Ng's article sumW is ΔW and sumb is Δb.
    sumW = [ zeros(sizes[2], sizes[1]),
            zeros(sizes[3], sizes[2]) ]
    sumb = [ zeros(sizes[2]), zeros(sizes[3]) ]

    # for each training example in the batch, use backprop
    # to compute the gradients and add them to the sum

    
    for i in batch
        x = trainx[i, :]
        y = trainy[i]
        ∇W, ∇b = backprop(W, b, x, y)
        for j in 1:2
            sumW[j] .+= ∇W[j]
            sumb[j] .+= ∇b[j]
        end
    end
    
    # make the update to the weights and biases and take a step
    # of gradient descent. note that we use the average gradient.

    for k in 1:length(W)
    
        W[k] = W[k] .- α * ((1/m) * sumW[k] + λ*W[k])
        b[k] = b[k] .- α * (1/m) * sumb[k]
    end

    
    # return the updated weights and biases. we also return the gradients
    return W, b, sumW, sumb
end


# classify the test data and compute the classification accuracy
function accuracy(W, b)
    ntest = length(testy)
    yhat = zeros(Int, ntest)
    for i in 1:ntest
        yhat[i] = classify(W, b, testx[i,:])
    end
    sum(testy .== yhat)/ntest # hit rate
end

# train the neural network using batch gradient descent.
# this is a driver function to repeatedly call GD().
# N = number of observations in the training data.
# m = batch size
# α = learning rate / step size
# λ = regularization parameter
function BGD(N, m, epochs; α=0.01, λ=0.01)
    # random initialization of the weights and biases
    d = Normal(0, 1)
    W = [ rand(d, sizes[2], sizes[1]),  # layer 1 to 2
          rand(d, sizes[3], sizes[2]) ] # layer 2 to 3
    b = [ rand(d, sizes[2]),   # layer 2
          rand(d, sizes[3]) ]  # layer 3
    ∇W = [ zeros(sizes[2], sizes[1]),  # layer 1 to 2
          zeros(sizes[3], sizes[2]) ] # layer 2 to 3
    ∇b = [ zeros(sizes[2]),   # layer 2
          zeros(sizes[3]) ]   # layer 3

    for e in epochs
        remaining = 1:N
        while length(remaining) > 0
            batch = sample(remaining, m, replace=false)
            remaining = setdiff(remaining, batch)
            W, b, ∇W, ∇b = GD(W,b,batch)
        end
        acc = accuracy(W,b)
    end

    return W, b, ∇W, ∇b
end

# some tuning parameters
N = length(trainy)
m = 25       # batch size
epochs = 10  # number of complete passes through the training data
α = 0.01     # learning rate / step size
λ = 0.01     # regularization parameter
W, b, ∇W, ∇b = BGD(N, m, epochs, α=α, λ=λ)

([[-0.9986062906572291 -1.6071732966272037 … -0.040289984671930436 -0.0541382873262382; -0.5926133644781323 0.9303213000912606 … 0.05083592049633394 -0.5397272071188509; … ; -0.0843610219990747 0.9426651021574727 … 0.529413990194043 -2.111582695432868; -0.6150401660464266 -0.9107616929153597 … 0.980329098883167 0.45714379354067725], [-7.134137084909282e-5 0.03323331656099597 … -0.20788117675945203 0.10651641861266287; -0.5641905267676229 -0.4136366046043286 … 1.2608057667501846 -1.923973329027189; … ; -1.5819186459448853 1.3989906673382813 … 2.0710203123520983 -2.321921426108096; -0.7334469141003682 0.3825677480073386 … 0.9087935352286151 -0.44773324929542957]], [[-2.820737269536844, -0.3303650089642246, -2.099063486302453, 2.134106389155389, -1.5991600134466966, -0.7286349167101922, -1.5840008523347413, 1.0610665290849683, -1.2582119894435178, -0.46447079373732114  …  -3.001110527504845, -1.3221895599392397, 1.0550639577100467, 0.23248384549234383, -1.6926080344442258, -0.152013001245

In [7]:


# unroll the weights and biases into a single vector.
# note this function will also work for unrolling the gradient.
# note that this is hard-coded for a 3-layer NN.
function unroll(W, b)
    vcat(vec(W[1]), vec(W[2]), vec(b[1]), vec(b[2]))
end

# given a single vector θ, reshape the parameters into the data
# structures that are used for backpropagation, that is, W and b, or
# ∇w and ∇b.  note that this is hard-coded for a 3-layer NN.
function reshape_params(θ)
    n1 = sizes[1]  # number of nodes in layer 1
    n2 = sizes[2]  # number of nodes in layer 2
    n3 = sizes[3]
    W1 = reshape(θ[1:(n2*n1)], n2, n1)
    W2 = reshape(θ[(n2*n1 + 1):(n2*n1 + n2*n3)], n3, n2)
    b1 = θ[(n2*n1 + n2*n3 + 1):(n2*n1 + n2*n3 + n2)]
    b2 = θ[(n2*n1 + n2*n3 + n2 + 1):length(θ)]
    W = [ W1, W2 ]
    b = [ b1, b2 ]
    return W, b
end

# evaluate the cost function for a batch of training examples
# θ is the unrolled vector of weights and biases.
# batch is the set of indices of the batch of training examples.
function J(θ, batch, λ)

    m = length(batch)
    sumJ = 0.0  # to accumulate the sum for the batch.
    # we need to pass W, b to feedforward, so we re-create W, b from θ
    W, b = reshape_params(θ)
    for i in batch
        x = trainx[i, :]  
        y = trainy[i]
        a, _ = feedforward(W, b, x)  # Calculate activations
        sumJ += (1/2) * sum((a[end] .- digit2vector(y)).^2)
    end
         

        # accumulate the cost function 
    ans = 0.0
    for l in 1:length(W)
        ans += sum(W[l] .^ 2)
    end

    func = (1/m) * sumJ + (λ/2) * ans
    return func

    # return the cost function. note that the regularization term only
    # applies to the weights, not the biases
    
end



# create the ith basis vector
function e(i)
    e = zeros(sizes[2]*sizes[1] + sizes[3]*sizes[2] + sizes[2] + sizes[3])
    e[i] = 1
    return e
end

θplus(v, i; ϵ=1e-4) = v .+ ϵ*e(i)
θminus(v, i; ϵ=1e-4) = v .- ϵ*e(i)

# compute the difference between the ith element of the gradient as
# computed from backpropagation (this is ∇θ[i]) and the approximation of
# the ith element of the gradient as obtained from finite differencing.
# the idea is to see if the backpropagation code is correctly computing
# the gradient of the cost function.
function compare1(i, θ, ∇θ, batch, λ; ϵ=1e-4)
    # i is the ith element of the unrolled gradient θ,
    ∇θ[i] - ( J(θplus(θ, i, ϵ=ϵ), batch, λ) - J(θminus(θ, i, ϵ=ϵ), batch, λ) )/(2*ϵ)
end

# compare each element of the gradient as computed from
# backpropagation to its estimate as obtained from finite
# differencing.
function compare(W, b, ∇W, ∇b, λ)
    θ = unroll(W, b)
    ∇θ = unroll(∇W, ∇b)
    m = length(trainy)

    batch = rand(1:m, 5000)
    tolerance = 0.001
    number_components_exceeding_tolerance = 0

    for i in 1:200
        finite_diff = compare1(i, θ, ∇θ, batch, λ)  
        if abs(finite_diff) > tolerance
            number_components_exceeding_tolerance += 1
            println("Component $i exceeds the tolerance: $finite_diff")
        end
    end

    println("$number_components_exceeding_tolerance components exceeded the tolerance of $tolerance")
end

compare(W, b, ∇W, ∇b, λ)


Component 1 exceeds the tolerance: 0.16637921253544513
Component 2 exceeds the tolerance: 0.16231928346695562
Component 3 exceeds the tolerance: 0.14638324427647725
Component 4 exceeds the tolerance: 0.14542721620199367
Component 5 exceeds the tolerance: 0.16146677407942783
Component 6 exceeds the tolerance: 0.153695782549163
Component 7 exceeds the tolerance: 0.1654202234872731
Component 8 exceeds the tolerance: 0.16795305822921486
Component 9 exceeds the tolerance: 0.14934543903262815
Component 10 exceeds the tolerance: 0.14695871070654115
Component 11 exceeds the tolerance: 0.15543587250573304
Component 12 exceeds the tolerance: 0.15678661950442924
Component 13 exceeds the tolerance: 0.15054264513577104
Component 14 exceeds the tolerance: 0.1514746276624795
Component 15 exceeds the tolerance: 0.1516154129658947
Component 16 exceeds the tolerance: 0.15370173860259123
Component 17 exceeds the tolerance: 0.1527695527185518
Component 18 exceeds the tolerance: 0.150106398043987
Component